# Creating and Valuing a CDS Contract

In [1]:
import pandas as pd
import QuantLib as ql
from devlib.market.curves.overnight_index_curves import Sofr

In [2]:
from nemesis.utils import *
from nemesis.products.rates import *
from nemesis.products.credit import *

####################################################################
#  NEMESIS ALPHA Version 0.1.0 - This build: 24 Jan 2025 at 10:42 #
####################################################################



## Build IR Curve

In [3]:
today = ql.Date(1,11,2024)
ql.Settings.instance().evaluationDate = today

mkt_file_path = './data/ICVS531_curve_data_20241101.xlsx'
swap_mkt_data = pd.read_excel(mkt_file_path, sheet_name='swap')
discount_curve = Sofr(today, swap_mkt_data=swap_mkt_data)

In [4]:
value_dt = Date(1,11,2024)
ois_curve = QLCurve(value_dt, discount_curve, dc_type=DayCountTypes.ACT_360, interp_type=InterpTypes.LINEAR_ZERO_RATES)

# Build CDS Curve

In [5]:
effective_dt = value_dt
settle_dt = effective_dt

In [6]:
cds_type = 'single_name'
mkt_file_path = './data/credit_curve_data_20241101.xlsx'
cds_data = pd.read_excel(mkt_file_path, sheet_name=cds_type)

In [7]:
cdss = [CDS(settle_dt, c["Tenor"], c["Spread"]/10000) for _, c in cds_data.iterrows()]

In [8]:
recovery_rate = 0.40

In [9]:
issuer_curve = CDSCurve(value_dt, cdss, ois_curve, recovery_rate)

In [10]:
print(issuer_curve)

OBJECT TYPE: CDSCurve
TIME,SURVIVAL_PROBABILITY
 0.0000000,  1.0000000
 0.6328767,  0.9967851
 1.1342466,  0.9936395
 2.1342466,  0.9851764
 3.1342466,  0.9742045
 4.1369863,  0.9608220
 5.1369863,  0.9427504
 7.1369863,  0.9023666
10.1397260,  0.8389955


# Valuation

In [26]:
cds_contract = CDS(
    step_in_dt=value_dt,
    maturity_dt_or_tenor="10Y",
    running_cpn=0.05,
    notional=1e7,
    long_protect=True,
    freq_type=FrequencyTypes.QUARTERLY,
    dc_type=DayCountTypes.ACT_360,
    cal_type=CalendarTypes.CHINA,
    bd_type=BusDayAdjustTypes.FOLLOWING,
    dg_type=DateGenRuleTypes.BACKWARD
)

In [42]:
cds_contract.accrual_end_dts

[19-DEC-2024,
 19-MAR-2025,
 19-JUN-2025,
 21-SEP-2025,
 21-DEC-2025,
 19-MAR-2026,
 21-JUN-2026,
 20-SEP-2026,
 20-DEC-2026,
 21-MAR-2027,
 20-JUN-2027,
 19-SEP-2027,
 19-DEC-2027,
 19-MAR-2028,
 19-JUN-2028,
 19-SEP-2028,
 19-DEC-2028,
 19-MAR-2029,
 19-JUN-2029,
 19-SEP-2029,
 19-DEC-2029,
 19-MAR-2030,
 19-JUN-2030,
 19-SEP-2030,
 19-DEC-2030,
 19-MAR-2031,
 19-JUN-2031,
 21-SEP-2031,
 21-DEC-2031,
 21-MAR-2032,
 20-JUN-2032,
 19-SEP-2032,
 19-DEC-2032,
 20-MAR-2033,
 19-JUN-2033,
 19-SEP-2033,
 19-DEC-2033,
 19-MAR-2034,
 19-JUN-2034,
 19-SEP-2034,
 20-DEC-2034]

In [29]:
spd = cds_contract.par_spread(settle_dt, issuer_curve, recovery_rate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD   96.75997 bp


In [30]:
v = cds_contract.value(settle_dt, issuer_curve, recovery_rate)

In [31]:
dirty_pv = v['dirty_pv'] 
clean_pv = v['clean_pv']

In [32]:
print("DIRTY VALUE %12.2f"% dirty_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

DIRTY VALUE  -3281419.76
CLEAN VALUE  -3223086.42


In [33]:
cleanp = cds_contract.clean_price(settle_dt, issuer_curve, recovery_rate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE   132.230864


In [34]:
accrued_days = cds_contract.accrued_days()
print("ACCRUED_DAYS", accrued_days)

ACCRUED_DAYS 42.0


In [35]:
accrued_interest = cds_contract.accrued_interest()
print("ACCRUED_COUPON", accrued_interest)

ACCRUED_COUPON -58333.33333333334


In [36]:
prot_pv = cds_contract.prot_leg_pv(settle_dt, issuer_curve, recovery_rate)
print("prot_PV", prot_pv)

prot_PV 773399.7261430764


In [37]:
premPV = cds_contract.premium_leg_pv(settle_dt, issuer_curve, recovery_rate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 4054819.4817989925


In [38]:
rpv01 = cds_contract.risky_pv01(settle_dt, issuer_curve)
print("DIRTY_RPV01", rpv01['dirty_rpv01'])
print("CLEAN_RPV01", rpv01['clean_rpv01'])

DIRTY_RPV01 8.109638963597984
CLEAN_RPV01 7.992972296931318


In [39]:
cds_contract.risky_pv01(settle_dt, issuer_curve)

{'dirty_rpv01': np.float64(8.109638963597984),
 'clean_rpv01': np.float64(7.992972296931318)}

## Risk Measures

In [40]:
credit_dv01 = cds_contract.credit_dv01(settle_dt, issuer_curve, recovery_rate)

TypeError: cannot pickle 'SwigPyObject' object

In [ ]:
credit_dv01

-650.5636208889919

Copyright (c) 2020 Dominic O'Kane